In [0]:
# %pip install databricks-sdk
%pip install --upgrade databricks-vectorsearch pydantic mlflow
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys, os
sys.path.append(os.path.abspath('..'))
from configs.project import ProjectConfig

from databricks.vector_search.client import VectorSearchClient
from datetime import timedelta
import time

In [0]:
import yaml

with open("../configs/project.yml", "r") as file:
    data = yaml.safe_load(file)

projectConfig = ProjectConfig(**data)
projectConfig.model_dump()

In [0]:
vector_search_key = list(projectConfig.vector_search_attributes.keys())[0]
globals().update(projectConfig.vector_search_attributes[vector_search_key])
# vector_search_endpoint_name = projectConfig.vector_search_attributes[vector_search_key]['vector_search_endpoint_name']
# embedding_model_endpoint_name = projectConfig.vector_search_attributes[vector_search_key]['embedding_model_endpoint_name']
# source_table_name = projectConfig.vector_search_attributes[vector_search_key]['source_table_name']
# index_name = projectConfig.vector_search_attributes[vector_search_key]['index_name']
# primary_key = projectConfig.vector_search_attributes[vector_search_key]['primary_key']
# embedding_source_column = projectConfig.vector_search_attributes[vector_search_key]['embedding_source_column']


In [0]:
vsc = VectorSearchClient(disable_notice=True)
# ep = vsc.get_endpoint(name=endpoint_name)
index = vsc.get_index(endpoint_name, index_name)

In [0]:
results = index.similarity_search(
  query_text="What was the last earnings results from American Express",
  columns=spark.table(source_table_name).columns,
  num_results=1
)

if results["result"]["row_count"] >0:
  display(results["result"]["data_array"])
else:
  print("No records")
